## Maximum path sum I

Problem 18

By starting at the top of the triangle below and moving to adjacent numbers on the row below, the maximum total from top to bottom is 23.

3
7 4
2 4 6
8 5 9 3

That is, 3 + 7 + 4 + 9 = 23.

Find the maximum total from top to bottom of the triangle below:

75  
95 64  
17 47 82  
18 35 87 10  
20 04 82 47 65  
19 01 23 75 03 34  
88 02 77 73 07 63 67  
99 65 04 28 06 16 70 92  
41 41 26 56 83 40 80 70 33  
41 48 72 33 47 32 37 16 94 29  
53 71 44 65 25 43 91 52 97 51 14  
70 11 33 28 77 73 17 78 39 68 17 57  
91 71 52 38 17 14 91 43 58 50 27 29 48  
63 66 04 68 89 53 67 30 73 16 69 87 40 31  
04 62 98 27 23 09 70 98 73 93 38 53 60 04 23  

NOTE: As there are only 16384 routes, it is possible to solve this problem by trying every route. However, Problem 67, is the same challenge with a triangle containing one-hundred rows; it cannot be solved by brute force, and requires a clever method! ;o)


Link: https://projecteuler.net/problem=18

Date solved:  
2022/03/08

In [1]:
# imports
import sys
sys.path.append('../../modules')


In [12]:
# %%timeit # results = 77.2 ms ± 1.71 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


class Node:
    
    def __init__(self,value=None):
        self.value = value
        self.nexts = []

    def add_next(self,next):
        self.nexts.append(next)

    def __repr__(self):
        return str(self.value)

class Graph:

    def __init__(self,nodes = None):
        self.nodes = nodes if nodes else []

    def add_node(self,node):
        self.nodes.append(node)

    def get_edges(self):
        edges = []
        for node in self.nodes:
            edges += [(node,next) for next in node.nexts]
        return edges
    
    def repr(self):
        output = ''
        for (n1,n2) in self.get_eges():
            output += f'{n1} -> {n2}\n'
        return output


n1 = Node(1)
n2 = Node(2)
n3 = Node(3)
n4 = Node(4)
n1.add_next(n2)
n1.add_next(n3)
n2.add_next(n4)
n3.add_next(n4)

g = Graph([n1,n2,n3,n4])

g.get_edges()


[(1, 2), (1, 3), (2, 4), (3, 4)]

## ANSWER = 142913828922